High Value Customer Identification (Insiders)

In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime
import requests

In [2]:
!pip list --format=freeze > requirements.txt

# Multiple products

In [14]:
url = 'https://www2.hm.com/en_us/men/products/jeans.html'

headers = {"User-Agent" : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'}

page = requests.get( url, headers=headers )

soup = BeautifulSoup( page.text, "html.parser")

total_item = soup.find_all( 'h2', class_='load-more-heading' )[0].get( 'data-total' )
total_item

'90'

In [15]:
page_number = np.round( int( total_item ) / 36 )
page_number

2.0

In [16]:
urlfull = url + '?page-size=' + str( int( page_number*36 ) )
urlfull

'https://www2.hm.com/en_us/men/products/jeans.html?page-size=72'

In [17]:
urlfull = 'https://www2.hm.com/en_us/men/products/jeans.html'

headers = {"User-Agent" : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'}

page = requests.get( url, headers=headers )

soup = BeautifulSoup( page.text, "html.parser")

total_item = soup.find_all( 'h2', class_='load-more-heading' )[0].get( 'data-total' )
total_item

'90'

In [18]:
products = soup.find( 'ul', class_='products-listing small' )

In [19]:
product_list = products.find_all( "article", class_="hm-product-item")

product_id = [ p.get( 'data-articlecode') for p in product_list]

product_category = [ p.get( 'data-category') for p in product_list]

list_name = products.find_all( "a", class_ = 'link')

product_name = [ p.get_text() for p in list_name]

price_list = products.find_all( "span", class_ = 'price regular')

product_price = [ p.get_text() for p in price_list]

In [20]:
df_products = pd.DataFrame( [product_id, product_category, product_name, product_price] ).T
df_products.columns = ['product_id', 'product_category', 'product_name', 'product_price']
df_products['scrapy_datetime'] = datetime.now().strftime( '%Y-%m-%d %H:%M:%S' )
#df_products['style_id'] = df_products['product_id'].str[:-3]
#df_products['color_id'] = df_products['product_id'].str[-3:]
df_products.head()

,product_id,product_category,product_name,product_price,scrapy_datetime
0,1024256001,men_jeans_slim,Slim Jeans,$ 19.99,2022-11-30 05:56:05
1,0985159001,men_jeans_skinny,Skinny Jeans,$ 24.99,2022-11-30 05:56:05
2,1008549001,men_jeans_regular,Regular Jeans,$ 24.99,2022-11-30 05:56:05
3,1024256003,men_jeans_slim,Slim Jeans,$ 19.99,2022-11-30 05:56:05
4,0690449022,men_jeans_ripped,Skinny Jeans,$ 39.99,2022-11-30 05:56:05


In [21]:
headers = {"User-Agent" : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'}

df_details = pd.DataFrame()

#unique columns for all products
aux = []

cols = ['Additional material information','Art. No.','Care instructions','Collection','Composition','Concept','Description','Fit','Imported','Material','Size']
df_pattern = pd.DataFrame( columns = cols)

for i in df_products['product_id'].unique():

    url = f'https://www2.hm.com/en_us/productpage.{i}.html'

    page = requests.get( url, headers=headers )

    soup = BeautifulSoup( page.text , 'html.parser' )

    product_list = soup.find_all('div', class_='details-attributes-list-item')


    # remove empty strings
    product_composition = [list(filter(None, p.get_text().split('\n'))) for p in product_list]


    # list to dataframe
    data_composition = pd.DataFrame(product_composition).T


    # data aux
    data_composition_aux = data_composition.iloc[0]


    # rename df columns
    data_composition.columns = data_composition_aux

    aux = aux + data_composition.columns.tolist()

    aux = [i for i in aux if "messages." not in i]

    # forward fillna
    data_composition = data_composition.iloc[1:].fillna(method='ffill')

    data_composition = pd.concat( [df_pattern, data_composition], axis=0)


    # drop duplicates
    data_composition = data_composition.drop_duplicates(subset=['Fit', 'Composition', 'Art. No.'], keep='first', inplace=False, ignore_index=False)


    # only selected columns
    data_composition = data_composition[cols].copy()

    data_composition.columns = data_composition.columns.str.lower()

    data_composition['style_id'] = data_composition['art. no.'].str[:-3]

    data_composition['color_id'] = data_composition['art. no.'].str[-3:]



    # Produto Final
    df_details = pd.concat( [df_details, data_composition] , axis=0)

In [22]:
df_details.head(3)

,additional material information,art. no.,care instructions,collection,composition,concept,description,fit,imported,material,size,style_id,color_id
1,NaN,1024256001,Only non-chlorine bleach when needed,NaN,"Shell: Cotton 99%, Spandex 1%",DENIM,Black,Slim fit,Yes,Denim,NaN,1024256,001
2,NaN,1024256001,Medium iron,NaN,"Pocket lining: Polyester 65%, Cotton 35%",DENIM,Solid-color,Slim fit,Yes,Denim,NaN,1024256,001
1,Recycled cotton 20%,0985159001,Only non-chlorine bleach when needed,NaN,"Shell: Cotton 99%, Spandex 1%",DENIM,Black,Skinny fit,Yes,Denim,NaN,0985159,001


In [23]:
df_products2 = df_products.merge(df_details, left_on='product_id', right_on='art. no.', how='left', validate='one_to_many')
print( df_products.shape, df_products2.shape)
df_products2.head(3)

(36, 5) (68, 18)


,product_id,product_category,product_name,product_price,scrapy_datetime,additional material information,art. no.,care instructions,collection,composition,concept,description,fit,imported,material,size,style_id,color_id
0,1024256001,men_jeans_slim,Slim Jeans,$ 19.99,2022-11-30 05:56:05,NaN,1024256001,Only non-chlorine bleach when needed,NaN,"Shell: Cotton 99%, Spandex 1%",DENIM,Black,Slim fit,Yes,Denim,NaN,1024256,001
1,1024256001,men_jeans_slim,Slim Jeans,$ 19.99,2022-11-30 05:56:05,NaN,1024256001,Medium iron,NaN,"Pocket lining: Polyester 65%, Cotton 35%",DENIM,Solid-color,Slim fit,Yes,Denim,NaN,1024256,001
2,0985159001,men_jeans_skinny,Skinny Jeans,$ 24.99,2022-11-30 05:56:05,Recycled cotton 20%,0985159001,Only non-chlorine bleach when needed,NaN,"Shell: Cotton 99%, Spandex 1%",DENIM,Black,Skinny fit,Yes,Denim,NaN,0985159,001


In [24]:
df_products2.to_csv("df_products.csv", index=False)